In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/202411-baywheels-tripdata.csv')

In [ ]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,65334AA1F43ACF70,electric_bike,2024-11-22 07:48:51.493,2024-11-22 08:09:05.683,Precita Park,SF-S24,22nd St Caltrain Station,SF-P30,37.747300,-122.411403,37.757704,-122.391885,member
1,557AE000ACD43DA3,electric_bike,2024-11-08 23:37:21.975,2024-11-08 23:41:32.349,Masonic Ave at Turk St,SF-I16,Lyon St at Fell St,SF-K17,37.779047,-122.447291,37.773562,-122.442571,member
2,49BDD1476E9B3518,electric_bike,2024-11-06 08:22:02.880,2024-11-06 08:38:44.914,Masonic Ave at Turk St,SF-I16,19th St at Florida St,SF-O25-1,37.779047,-122.447291,37.760447,-122.410807,member
3,BDABE3DC05001C58,electric_bike,2024-11-16 17:00:40.752,2024-11-16 17:06:52.851,5th St at Virginia St,SJ-P10,Fountain Alley at S 2nd St,SJ-M10-4,37.325998,-121.877120,37.336188,-121.889277,member
4,2AF447DEE5822972,electric_bike,2024-11-16 18:01:29.950,2024-11-16 18:15:56.591,Fountain Alley at S 2nd St,SJ-M10-4,5th St at Virginia St,SJ-P10,37.336188,-121.889277,37.325998,-121.877120,member


Dimensional Tables

In [ ]:
import pandas as pd
import numpy as np
# Create date dimension from start and end dates
date_dim = df[['started_at', 'ended_at']].drop_duplicates().reset_index(drop=True)

# Convert columns to datetime
date_dim['started_at'] = pd.to_datetime(date_dim['started_at'])
date_dim['ended_at'] = pd.to_datetime(date_dim['ended_at'])

# Add date attributes
date_dim['date_key'] = range(1, len(date_dim) + 1)
date_dim['pick_hour'] = date_dim['started_at'].dt.hour
date_dim['pick_day'] = date_dim['started_at'].dt.day
date_dim['pick_month'] = date_dim['started_at'].dt.month
date_dim['pick_year'] = date_dim['started_at'].dt.year
date_dim['pick_weekday'] = date_dim['started_at'].dt.weekday
date_dim['pick_dayname'] = date_dim['started_at'].dt.day_name()
date_dim['drop_hour'] = date_dim['ended_at'].dt.hour
date_dim['drop_day'] = date_dim['ended_at'].dt.day
date_dim['drop_month'] = date_dim['ended_at'].dt.month
date_dim['drop_year'] = date_dim['ended_at'].dt.year
date_dim['drop_weekday'] = date_dim['ended_at'].dt.weekday
date_dim['drop_dayname'] = date_dim['ended_at'].dt.day_name()


# First convert datetime columns in both dataframes
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
date_dim['started_at'] = pd.to_datetime(date_dim['started_at'])
date_dim['ended_at'] = pd.to_datetime(date_dim['ended_at'])


# Add derived columns
date_dim['is_weekend'] = date_dim['pick_weekday'].isin([5, 6])
date_dim['is_peak_hour'] = date_dim['pick_hour'].isin([7, 8, 9, 16, 17, 18])

# Print preview of the date dimension
print("\nDATE DIMENSION PREVIEW")
print("=====================")
print(date_dim.head())


DATE DIMENSION PREVIEW
               started_at                ended_at  date_key  pick_hour  \
0 2024-11-22 07:48:51.493 2024-11-22 08:09:05.683         1          7   
1 2024-11-08 23:37:21.975 2024-11-08 23:41:32.349         2         23   
2 2024-11-06 08:22:02.880 2024-11-06 08:38:44.914         3          8   
3 2024-11-16 17:00:40.752 2024-11-16 17:06:52.851         4         17   
4 2024-11-16 18:01:29.950 2024-11-16 18:15:56.591         5         18   

   pick_day  pick_month  pick_year  pick_weekday pick_dayname  drop_hour  \
0        22          11       2024             4       Friday          8   
1         8          11       2024             4       Friday         23   
2         6          11       2024             2    Wednesday          8   
3        16          11       2024             5     Saturday         17   
4        16          11       2024             5     Saturday         18   

   drop_day  drop_month  drop_year  drop_weekday drop_dayname  is_weekend 

In [ ]:
# 2. Create Start Station Dimension
start_stations = df[['start_station_id', 'start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
start_stations = start_stations.rename(columns={
    'start_lat': 'start_latitude',
    'start_lng': 'start_longitude'
})
start_stations['start_station_key'] = np.arange(len(start_stations))

# Print preview of the station dimension
print("\nSTART STATION DIMENSION PREVIEW")
print("========================")
print(start_stations.head())



START STATION DIMENSION PREVIEW
  start_station_id          start_station_name  start_latitude  \
0           SF-S24                Precita Park       37.747300   
1           SF-I16      Masonic Ave at Turk St       37.779047   
3           SJ-P10       5th St at Virginia St       37.325998   
4         SJ-M10-4  Fountain Alley at S 2nd St       37.336188   
5           BK-F10     Derby St at College Ave       37.861760   

   start_longitude  start_station_key  
0      -122.411403                  0  
1      -122.447291                  1  
3      -121.877120                  2  
4      -121.889277                  3  
5      -122.253677                  4  


In [ ]:
# 3. Create End Station Dimension
end_stations = df[['end_station_id', 'end_station_name', 'end_lat', 'end_lng']].drop_duplicates()
end_stations = end_stations.rename(columns={
    'end_lat': 'end_latitude',
    'end_lng': 'end_longitude'
})
end_stations['end_station_key'] = np.arange(len(end_stations))

# Print preview of the station dimension
print("\nEND STATION DIMENSION PREVIEW")
print("========================")
print(end_stations.head())


END STATION DIMENSION PREVIEW
  end_station_id            end_station_name  end_latitude  end_longitude  \
0         SF-P30    22nd St Caltrain Station     37.757704    -122.391885   
1         SF-K17          Lyon St at Fell St     37.773562    -122.442571   
2       SF-O25-1       19th St at Florida St     37.760447    -122.410807   
3       SJ-M10-4  Fountain Alley at S 2nd St     37.336188    -121.889277   
4         SJ-P10       5th St at Virginia St     37.325998    -121.877120   

   end_station_key  
0                0  
1                1  
2                2  
3                3  
4                4  


In [ ]:
# 4. Create Bike Dimension
bike_dim = df[['ride_id', 'rideable_type']].drop_duplicates()
bike_dim['bike_key'] = np.arange(len(bike_dim))
bike_dim['is_electric'] = bike_dim['rideable_type'] == 'electric_bike'
bike_dim['bike_category'] = bike_dim['rideable_type'].map({
    'electric_bike': 'electric',
    'classic_bike': 'classic',
})

# Print preview of the bike dimension
print("\nBIKE DIMENSION PREVIEW")
print("=====================")
print(bike_dim.head())


BIKE DIMENSION PREVIEW
            ride_id  rideable_type  bike_key  is_electric bike_category
0  65334AA1F43ACF70  electric_bike         0         True      electric
1  557AE000ACD43DA3  electric_bike         1         True      electric
2  49BDD1476E9B3518  electric_bike         2         True      electric
3  BDABE3DC05001C58  electric_bike         3         True      electric
4  2AF447DEE5822972  electric_bike         4         True      electric


In [ ]:
# 5. Create Rider Dimension
rider_dim = df[['member_casual']].drop_duplicates()
rider_dim['rider_key'] = np.arange(len(rider_dim))
rider_dim['is_member'] = rider_dim['member_casual'] == 'member'
rider_dim['rider_type'] = rider_dim['member_casual'].map({
    'member': 'subscriber',
    'casual': 'casual'
})

# Print the first few rows
print(rider_dim.head())

   member_casual  rider_key  is_member  rider_type
0         member          0       True  subscriber
10        casual          1      False      casual


In [ ]:
# 6. Create Distance Dimension
# First create the distance category column
distance_ranges = [0, 1, 2, 5, 10, float('inf')]
distance_labels = ['Very Short', 'Short', 'Medium', 'Long', 'Very Long']

# Calculate distance in km using the latitude and longitude
df['distance_km'] = np.sqrt(
    (df['end_lat'] - df['start_lat'])**2 +
    (df['end_lng'] - df['start_lng'])**2
) * 111  # Rough conversion to kilometers

# Create distance categories
df['distance_category'] = pd.cut(
    df['distance_km'],
    bins=distance_ranges,
    labels=distance_labels
)

# Now create the distance dimension
distance_dim = df[['distance_category', 'distance_km']].drop_duplicates()
distance_dim['distance_key'] = np.arange(len(distance_dim))

# Verify the dimension was created
print("Distance Dimension Preview:")
print(distance_dim.head())
print("\nShape:", distance_dim.shape)

Distance Dimension Preview:
  distance_category  distance_km  distance_key
0            Medium     2.455093             0
1        Very Short     0.803189             1
2            Medium     4.545658             2
3             Short     1.760723             3
5            Medium     2.419656             4

Shape: (47058, 3)


Fact Table

In [ ]:
print("DataFrame shapes:")
print("Main df:", df.shape)
print("Date dimension:", date_dim.shape)
print("Start stations:", start_stations.shape)
print("End stations:", end_stations.shape)
print("Bike dimension:", bike_dim.shape)
print("Rider dimension:", rider_dim.shape)
print("Distance dimension:", distance_dim.shape)

# Also check column types
print("\nMain df dtypes:")
print(df.dtypes)

DataFrame shapes:
Main df: (261311, 15)
Date dimension: (261311, 17)
Start stations: (11200, 5)
End stations: (948, 5)
Bike dimension: (261311, 5)
Rider dimension: (2, 4)
Distance dimension: (47058, 3)

Main df dtypes:
ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
distance_km                  float64
distance_category           category
dtype: object


In [ ]:
fact_table = df.copy()

# Calculate ride duration in minutes
fact_table['ride_duration_minutes'] = (fact_table['ended_at'] - fact_table['started_at']).dt.total_seconds() / 60

# Merge with date dimension
fact_table = fact_table.merge(
    date_dim[['date_key', 'started_at', 'ended_at']],
    on=['started_at', 'ended_at'],
    how='left'
)

# Merge with start station dimension
fact_table = fact_table.merge(
    start_stations[['start_station_key', 'start_station_id', 'start_station_name',
                   'start_latitude', 'start_longitude']],
    left_on=['start_station_id', 'start_station_name', 'start_lat', 'start_lng'],
    right_on=['start_station_id', 'start_station_name', 'start_latitude', 'start_longitude'],
    how='left'
)

# Merge with end station dimension
fact_table = fact_table.merge(
    end_stations[['end_station_key', 'end_station_id', 'end_station_name',
                 'end_latitude', 'end_longitude']],
    left_on=['end_station_id', 'end_station_name', 'end_lat', 'end_lng'],
    right_on=['end_station_id', 'end_station_name', 'end_latitude', 'end_longitude'],
    how='left'
)

# Merge with bike dimension
fact_table = fact_table.merge(
    bike_dim[['bike_key', 'ride_id', 'rideable_type']],
    on=['ride_id', 'rideable_type'],
    how='left'
)

# Merge with rider dimension
fact_table = fact_table.merge(
    rider_dim[['rider_key', 'member_casual']],
    on=['member_casual'],
    how='left'
)

# Merge with distance dimension
fact_table = fact_table.merge(
    distance_dim[['distance_key', 'distance_km']],
    on=['distance_km'],
    how='left'
)

# Drop the redundant columns
columns_to_drop = [
    'start_lat', 'start_lng', 'end_lat', 'end_lng',
    'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude'
]
fact_table = fact_table.drop(columns=columns_to_drop, errors='ignore')

# Select only the necessary columns for the fact table
fact_table = fact_table[[
    'ride_id',
    'date_key',
    'start_station_key',
    'end_station_key',
    'bike_key',
    'rider_key',
    'distance_key',
    'ride_duration_minutes',
    'distance_km'
]]

# Verify no missing values in key columns
print("\nMissing values in fact table:")
print(fact_table.isnull().sum())
fact_table.head()


Missing values in fact table:
ride_id                   0
date_key                  0
start_station_key         0
end_station_key           0
bike_key                  0
rider_key                 0
distance_key              0
ride_duration_minutes     0
distance_km              69
dtype: int64


,ride_id,date_key,start_station_key,end_station_key,bike_key,rider_key,distance_key,ride_duration_minutes,distance_km
0,65334AA1F43ACF70,1,0,0,0,0,0,20.236500,2.455093
1,557AE000ACD43DA3,2,1,1,1,0,1,4.172900,0.803189
2,49BDD1476E9B3518,3,1,2,2,0,2,16.700567,4.545658
3,BDABE3DC05001C58,4,2,3,3,0,3,6.201650,1.760723
4,2AF447DEE5822972,5,3,4,4,0,3,14.444017,1.760723


In [ ]:
fact_table.shape

(261311, 9)

In [ ]:
{"date_dim":date_dim.to_dict(orient="dict"),
    "start_stations":start_stations.to_dict(orient="dict"),
    "end_stations":end_stations.to_dict(orient="dict"),
    "bike_dim":bike_dim.to_dict(orient="dict"),
    "rider_dim":rider_dim.to_dict(orient="dict"),
    "distance_dim":distance_dim.to_dict(orient="dict"),
    "fact_table":fact_table.to_dict(orient="dict")}


{'date_dim': {'started_at': {0: Timestamp('2024-11-22 07:48:51.493000'),
   1: Timestamp('2024-11-08 23:37:21.975000'),
   2: Timestamp('2024-11-06 08:22:02.880000'),
   3: Timestamp('2024-11-16 17:00:40.752000'),
   4: Timestamp('2024-11-16 18:01:29.950000'),
   5: Timestamp('2024-11-06 20:23:39.684000'),
   6: Timestamp('2024-11-29 17:23:35.232000'),
   7: Timestamp('2024-11-04 19:31:02.639000'),
   8: Timestamp('2024-11-09 22:04:39.473000'),
   9: Timestamp('2024-11-01 10:23:08.747000'),
   10: Timestamp('2024-11-29 12:21:39.806000'),
   11: Timestamp('2024-11-29 12:31:49.895000'),
   12: Timestamp('2024-11-28 15:12:09.806000'),
   13: Timestamp('2024-11-26 14:47:14.369000'),
   14: Timestamp('2024-11-16 17:23:12.727000'),
   15: Timestamp('2024-11-08 19:04:40.970000'),
   16: Timestamp('2024-11-02 12:46:32.821000'),
   17: Timestamp('2024-11-21 16:58:03.259000'),
   18: Timestamp('2024-11-09 08:37:59.696000'),
   19: Timestamp('2024-11-18 16:10:55.223000'),
   20: Timestamp('2024-1

# **GOOGLE BIG QUEREY**

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Also set up your project explicitly
from google.cloud import bigquery
import pandas as pd

# Set your project ID explicitly
project_id = 'my-lyft-project-444506'
client = bigquery.Client(project=project_id)

In [ ]:
def upload_single_table(table_name, table_data):
    try:
        # Convert dict to dataframe
        df = pd.DataFrame(table_data)

        # Define table reference
        table_ref = f"{project_id}.lyft_data_engineering.{table_name}"

        print(f"Attempting to upload {table_name} to {table_ref}")

        # Upload configuration
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE"
        )

        # Upload to BigQuery
        job = client.load_table_from_dataframe(
            df,
            table_ref,
            job_config=job_config
        )

        # Wait for the job to complete
        job.result()
        print(f"Successfully uploaded {table_name}")

    except Exception as e:
        print(f"Detailed error for {table_name}:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")

# Test with date_dim first
upload_single_table('date_dim', date_dim.to_dict(orient="dict"))

Attempting to upload date_dim to my-lyft-project-444506.lyft_data_engineering.date_dim
Successfully uploaded date_dim


In [ ]:
# Dictionary of all your tables
tables = {
    'date_dim': date_dim.to_dict(orient="dict"),
    'start_stations': start_stations.to_dict(orient="dict"),
    'end_stations': end_stations.to_dict(orient="dict"),
    'bike_dim': bike_dim.to_dict(orient="dict"),
    'rider_dim': rider_dim.to_dict(orient="dict"),
    'distance_dim': distance_dim.to_dict(orient="dict"),
    'fact_table': fact_table.to_dict(orient="dict")
}

# Upload each table
for table_name, table_data in tables.items():
    upload_single_table(table_name, table_data)

Attempting to upload date_dim to my-lyft-project-444506.lyft_data_engineering.date_dim
Successfully uploaded date_dim
Attempting to upload start_stations to my-lyft-project-444506.lyft_data_engineering.start_stations
Successfully uploaded start_stations
Attempting to upload end_stations to my-lyft-project-444506.lyft_data_engineering.end_stations
Successfully uploaded end_stations
Attempting to upload bike_dim to my-lyft-project-444506.lyft_data_engineering.bike_dim
Successfully uploaded bike_dim
Attempting to upload rider_dim to my-lyft-project-444506.lyft_data_engineering.rider_dim
Successfully uploaded rider_dim
Attempting to upload distance_dim to my-lyft-project-444506.lyft_data_engineering.distance_dim
Successfully uploaded distance_dim
Attempting to upload fact_table to my-lyft-project-444506.lyft_data_engineering.fact_table
Successfully uploaded fact_table


In [ ]:
# Test BigQuery connection
def test_connection():
    try:
        # List datasets in your project
        datasets = list(client.list_datasets())
        print("Connection successful!")
        print(f"Found {len(datasets)} datasets in project {project_id}")
        for dataset in datasets:
            print(f"- {dataset.dataset_id}")
    except Exception as e:
        print(f"Connection error: {str(e)}")

test_connection()

Connection successful!
Found 1 datasets in project my-lyft-project-444506
- lyft_data_engineering
